In [52]:
import pandas as pd
import os
from datetime import datetime, date
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

# This function converts given date to age
def age(born, days_present = True):
    today = date.today()
    if not days_present:
        return today.year - born.year - int(today.month < born.month)    
    return today.year - born.year - ((today.month, today.day) <
                                     (born.month, born.day))

## Import and preprocess OPENonOH demographics and statistical data
Get it into the same shape as the OpenAPS dataset (OpenAPS complete_patient_statistics.xlsx)

In [53]:
dir_ = "/home/reinhold/Daten/Paper_Datasets_Nov2022"
filenames = [
    r'FINAL n=75 statistics processed.xlsx', r'Demo+SRCO_Adults_Caregivers.xlsx', r'n=75_deduped_demographics.xlsx'] # Specify the file path

#columns_ = ["age","height", "weight", "TOR<70", "TIR", "TOR>180"] # "dailyInsulinUnits", "dailyBasalInsulinUnits", "dailyCarbs" missing
col_maps = {"TBR<70 [%]" : "TOR<70", "TIR [%]" : "TIR", "TAR>180 [%]" : "TOR>180"}
col_maps.update({'project_member_id_x': 'project_member_id', "age_x" : "age", 'height_cm_v2_x' : 'height_cm_v2', 'weight_kg_x' : 'weight_kg'})
col_maps.update({0.25 : "Q1", 0.5 : "Q2", 0.75 : "Q3", "year_of_birth" : "year", "month_of_birth" : "month"})

df_OPENonOH_0 = pd.read_excel(os.path.join(dir_, filenames[0]))
df_OPENonOH_1 = pd.read_excel(os.path.join(dir_, filenames[1]), sheet_name="n=75_deduped_compressed")
df_OPENonOH_2 = pd.read_excel(os.path.join(dir_, filenames[2]))

df = pd.merge(df_OPENonOH_0, df_OPENonOH_1, left_on='id', right_on='project_member_id', how='inner')
df.rename(columns=col_maps, inplace=True)
df['age'] = df.apply(lambda x: age(x, False), axis=1)
df['height'] = df['height_cm_v2']
df.loc[df['height'].isnull(), 'height'] = df.loc[df['height'].isnull(), 'height_inches_v2'] * 30.48 + df.loc[df['height'].isnull(), 'height_inches_2_v2'] * 2.54
df['weight'] = df['body_weight_kg_v2']
df.loc[df['weight'].isnull(), 'weight'] = df.loc[df['weight'].isnull(), 'body_weight_pounds_v2'] * 0.453592

df = pd.merge(df, df_OPENonOH_2, left_on='id', right_on='project_member_id', how='inner')
df.rename(columns=col_maps, inplace=True)
for exclusions in ["child", "_x", "_y"]:
    df.drop(columns=[c for c in df.columns if exclusions in str(c)], inplace=True)

print(len(df.columns), df.columns)
df.to_excel(os.path.join(dir_, "OPENonOH complete_patient_statistics.xlsx"))

29 Index(['id', 'interdaysd', 'interdaycv', 'bg_roc_stds', 'TOR<70', 'TIR',
       'TOR>180', 'POR', 'J_index', 'LGBI', 'HBGI', 'GMI', 'count', 'mean',
       'std', 'min', 'Q1', 'Q2', 'Q3', 'max', 'project_member_id', 'month',
       'year', 'height_cm_v2', 'age', 'height', 'weight', 'month', 'year'],
      dtype='object')


In [54]:
#df.info()
#df.head(2)
df.columns

Index(['id', 'interdaysd', 'interdaycv', 'bg_roc_stds', 'TOR<70', 'TIR',
       'TOR>180', 'POR', 'J_index', 'LGBI', 'HBGI', 'GMI', 'count', 'mean',
       'std', 'min', 'Q1', 'Q2', 'Q3', 'max', 'project_member_id', 'month',
       'year', 'height_cm_v2', 'age', 'height', 'weight', 'month', 'year'],
      dtype='object')